# Project 4


In [2]:
import xgboost as xgb

In [3]:
from xgboost import XGBClassifier

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [5]:
import pandas as pd
import numpy as np

#must ensure conda install py-xgboost
#import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\erict/nltk_data'
    - 'C:\\Users\\erict\\anaconda3\\envs\\learn-env\\nltk_data'
    - 'C:\\Users\\erict\\anaconda3\\envs\\learn-env\\share\\nltk_data'
    - 'C:\\Users\\erict\\anaconda3\\envs\\learn-env\\lib\\nltk_data'
    - 'C:\\Users\\erict\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


Loading Data

In [ ]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')


In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.info()

In [ ]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train_df.user_suggestion.values)


In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train_df.user_review.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

# Starting Models
- TF-IDF
- Logistic Regression

In [11]:
# Following features are common starting points and work well in most cases.
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [12]:
# Fit a simple Logistic Regression on TF-IDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)
bin_preds = clf.predict(xvalid_tfv)
# sklearn.metrics.log_loss(y_true, y_pred, *, eps=1e-15, normalize=True, sample_weight=None, labels=None)
from sklearn.metrics import log_loss, f1_score, accuracy_score

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))
#Note, we want to be careful about applying accuracy/f1 here because those are classification metrics, and only apply to the
#binary outputs of .predict.  
#We can apply the predict_proba to get actual probabilities (i.e. before the softmax step) that are possibly more
#descriptive of what the regression is doing.


logloss: 0.384 
f1 score: 0.879 
Accuracy score: 0.858 


Instead of using TF-IDF, we can also use word counts as features. This can be done easily using CountVectorizer from scikit-learn.

In [13]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [14]:
#checking out an arbitrary entry of that ctv
#xtrain_ctv[1,2]

In [15]:
# Fit a simple Logistic Regression on those Counts
clf = LogisticRegression(C=1.0, max_iter=500,)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)
bin_preds = clf.predict(xvalid_ctv)

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

logloss: 0.449 
f1 score: 0.872 
Accuracy score: 0.851 


Ok, worse loss, f1, and accuracy.

In [16]:
# Fitt a simple Naive Bayes on TF-IDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)
bin_preds = clf.predict(xvalid_tfv)

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

logloss: 0.400 
f1 score: 0.868 
Accuracy score: 0.836 


Wow!  Naive Bayes improved loss!     

Accuracy and F1 took a hit though, but the improvement in loss implies that the model is overall better at describing things.  

What happens when we use this model on counts data instead?

In [17]:
# Fit a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)
bin_preds = clf.predict(xvalid_ctv)

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

logloss: 1.286 
f1 score: 0.876 
Accuracy score: 0.859 


NB on counts did worse on loss, but held ground on F1 and Acc.

One more classic algorithm is SVMs. 

In order to improve performance on time, we will reduce the number of features from the TF-IDF using Singular Value Decomposition before applying SVM.

Also, it is better to standardize the data before applying SVMs.

In [18]:
# Apply SVD, with 120 components. 120-200 components are widely considered sufficient for SVM.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

Now time to fit the SVM.  This can take a while!

In [20]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we want probabilities in addition to the 0/1 scores
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)
bin_preds = clf.predict(xvalid_svd_scl)


print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

logloss: 0.368 
f1 score: 0.858 
Accuracy score: 0.835 


This is even better loss and good f1/acc.

That performed pretty well on logloss.  Neat!  

Of course, we couldn't stop without trying XGBoost!

In [23]:
# Fit a basic xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())
bin_preds = clf.predict(xvalid_tfv.tocsc())


print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

logloss: 0.393 
f1 score: 0.856 
Accuracy score: 0.831 


Strangely, XGBoost didn't do as well on TF-IDF features as logistic regression.

In [27]:
# Fitting a simple xgboost on tf-idf svd features with default parameters
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)
bin_preds = clf.predict(xvalid_svd_scl)

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

C:\Users\Eric\anaconda3\envs\learn-env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:56:55] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.411 
f1 score: 0.758 
Accuracy score: 0.707 


In [24]:
# Fit a basic xgboost on more common hyperparams for tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)
bin_preds = clf.predict(xvalid_svd)

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

C:\Users\Eric\anaconda3\envs\learn-env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:51:06] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.390 
f1 score: 0.847 
Accuracy score: 0.823 


On TF-IDF SVD unscaled features, XGBoost does not improve.  

What about on scaled features?

In [26]:
# Fit a basic xgboost on SCALED tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)
bin_preds = clf.predict(xvalid_svd_scl)

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))

C:\Users\Eric\anaconda3\envs\learn-env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:54:28] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.390 
f1 score: 0.847 
Accuracy score: 0.823 


And on Scaled TF-IDF SVD features, XGboost does very similarly.  

But wait!  We haven't done any tuning of hyperparameters yet.

# Improve all those models with some grid search optimization  
First, we can define a scoring metric for automation.

In [30]:
basic_scorer = metrics.make_scorer(log_loss, greater_is_better=False, needs_proba=True)

Since we applied svd and scaling to two different sets (TF-IDF and SVD), this seems like it has potential for a pipeline approach

In [31]:
# Initialize SVD
svd = TruncatedSVD()
    
# Initialize a standard scaler 
scl = preprocessing.StandardScaler()

# We will use logistic regression here.
lr_model = LogisticRegression()

# Create the Logistic Regression pipeline 
clf_lr = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

#Set up the parameter grid
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

In [32]:
# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=basic_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=4)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here because of CV, but probably even better integrity if we reserve Validation set
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 4 folds for each of 12 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  46 out of  48 | elapsed:  8.6min remaining:   22.2s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  8.6min finished
C:\Users\Eric\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\Users\Eric\anaconda3\envs\learn-env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_enc

[15:42:59] WARNING: ..\src\learner.cc:576: 
Parameters: { "lr__C", "lr__penalty", "svd__n_components" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[15:42:59] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best score: -0.401
Best parameters set:
	lr__C: 0.1
	lr__penalty: 'l1'
	svd__n_components: 120


 (results summary)

In [1]:
#Now try optimizing Naive Bayes

nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

NameError: name 'MultinomialNB' is not defined

# Pretrained Models

In [ ]:
# GloVe vectors from here:
http://www-nlp.stanford.edu/data/glove.840B.300d.zip

In [ ]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('../input/glove840b300dtxt/glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower().decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [ ]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]

In [ ]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

Check the performance of xgboost on glove features:`

In [ ]:
# Fit a simple xgboost on glove features
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)


print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

In [ ]:
# Fit a simple xgboost on glove features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)
bin_preds = clf.predict(xvalid_svd)

print ("logloss: %0.3f " % log_loss(yvalid, predictions))
print ("f1 score: %0.3f " % f1_score(yvalid, bin_preds))
print ("Accuracy score: %0.3f " % accuracy_score(yvalid, bin_preds))


In [ ]:
# a simple tuning of parameters can improve xgboost score on GloVe features!
#but we can tune even more to improve further

# Deep Learning  
LSTM and dense network
ADD GRU

In [ ]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [ ]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [ ]:
# create a 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=5, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

In [ ]:
'''You need to keep on tuning the parameters of the neural network, add more layers, 
increase dropout to get better results. Here, I'm just showing that its fast to implement and run and gets better 
result than xgboost without any optimization :)

To move further, i.e. with LSTMs we need to tokenize the text data'''

In [ ]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [ ]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

In [ ]:
'''We see that the score is now less than 0.5. I ran it for many epochs without stopping at the best but you can use early stopping to stop at the best iteration. How do I use early stopping?

well, pretty easy. let's compile the model again:'''

In [ ]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

In [ ]:
'''One question could be: why do i use so much dropout? Well, fit the model with no or little dropout and you will that it starts to overfit :)

Let's see if Bi-directional LSTM can give us better results. Its a piece of cake to do it with Keras :)'''

In [21]:
#set up apache spark system to do parallel processing?
#maybe Try to set up amazon resources - possibly overkill/not good for github
## see recommendation systems section - this isn't actually directly applicable - more for recs for a user based on other user's recs
# topic 39 - nlp obvs
#tf-idfs?
#nnetworks with
# word2vec (including gensim) see https://learning.flatironschool.com/courses/1890/pages/using-word2vec?module_item_id=261958
#GloVe for classification - (This is a very good pretrained model)
# word embeddings, 
#sequence models e.g. 
#RNN and 
#lstms, 
#grus, 
#gridsearch for best params,
#ensemble methods - e.g. xgboost on these models
#anything else that is in 
# https://www.kaggle.com/code/abhishek/approaching-almost-any-nlp-problem-on-kaggle/comments
'''tfidf
count features
logistic regression
naive bayes
svm
xgboost
grid search
word vectors
LSTM
GRU
Ensembling'''



'tfidf\ncount features\nlogistic regression\nnaive bayes\nsvm\nxgboost\ngrid search\nword vectors\nLSTM\nGRU\nEnsembling'